<a href="https://colab.research.google.com/github/Mayugumiii/UIT_DS_Challenge_Text_Reranking/blob/main/UIT_DS_Challenge_Text_Reranking_T_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install -U sentence-transformers
!pip install pyvi
!pip install rank-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=5c84ba4f59beaa9dbc3c58175d7973dd92aae49412a7ecc8ac4696c7cb319487
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 37.3 MB/s eta 0

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from rank_bm25 import BM25Okapi
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [31]:
#Load data
df = pd.read_excel('/content/drive/MyDrive/Dataset/final_data.xlsx')
passages = df.iloc[:, 0]
labels = df.iloc[:,1]
claims = df.iloc[:, 2]

In [39]:
# Load the PhoBERT model and tokenizer
model = AutoModel.from_pretrained("bert-base-multilingual-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [41]:
# Create a list of dictionaries to store the data for each claim
output_data = []
i = 0
for passage, label, claim in zip(passages, labels, claims):
    premise_sentences = sent_tokenize(passage)

    claim = str(claim)
    encoded_claim = tokenizer(claim, return_tensors='pt')

    #BM25 rerank top k
    tokenized_premise_sentences = [word.split(" ") for word in premise_sentences]
    tokenized_claim = claim.split(" ")
    bm25 = BM25Okapi(tokenized_premise_sentences)
    bm25_scores = bm25.get_scores(tokenized_claim)

    top_k_sentences = bm25.get_top_n(tokenized_claim, premise_sentences, n=3)
    encoded_premise = [tokenizer(sentence, return_tensors='pt') for sentence in top_k_sentences]

    # Embed the top k sentences and claim
    embedded_premise = [model(**sentence) for sentence in encoded_premise]
    embedded_claim = model(**encoded_claim)

    # Initialize variables for each iteration
    highest_similarity = -1.1
    best_premise_sentence = None

    # Tính cos sim từng câu trong top k để so sánh với claim
    for sentence, premise_embedding in zip(top_k_sentences, embedded_premise):
        similarity = F.cosine_similarity(embedded_claim.last_hidden_state.mean(dim=1),
                                         premise_embedding.last_hidden_state.mean(dim=1), dim=1)

        # Lưu trữ câu có cos sim cao nhất
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_premise_sentence = sentence

    # Create a dictionary for the current claim
    claim_data = {
        "context": passage,
        "claim" : claim,
        "verdict": label,
        "evidence": best_premise_sentence
    }

    # Append các claim vào list
    output_data.append(claim_data)
    print("Best Similarity:", claim)
    print("Best Premise Sentence:", best_premise_sentence)


# Lưu claim vào file
with open('output.json', 'w') as jsonfile:
    json.dump(output_data, jsonfile, indent=4)

Best Similarity: Cháu H.P.Đ tử vong vì bị chó cắn vào má
Best Premise Sentence: Bị chó nhà hàng xóm cắn vào gò má nhưng không tiêm vắc xin và huyết thanh kháng dại, gần một tháng sau, cháu bé 6 tuổi ở Quảng Trị tử vong.
Best Similarity: Thận bò hoạt động trong cơ thể ông Maurice suốt hơn một tháng cho thấy bước tiến quan trọng trong cấy ghép tạng động vật cho người.
Best Premise Sentence: MỸ - Thận lợn hoạt động trong cơ thể ông Maurice suốt hơn một tháng cho thấy bước tiến quan trọng trong cấy ghép tạng động vật cho người.
Best Similarity:  Bạn không nên cho kem đánh răng lên bàn chải còn khô vào buổi sáng
Best Premise Sentence: Bạn không nên cho kem đánh răng lên bàn chải còn khô.
Best Similarity: Không được hút thuốc tại nhà khách, nhà nghỉ.
Best Premise Sentence: Các địa điểm cấm hút thuốc lá: Cần biết để không bị phạt.
Best Similarity: Các chuyên gia tại TP.HCM không đề xuất thay đổi quy định liên quan đến việc chăm sóc, điều trị người bệnh vừa có rối loạn tâm thần vừa mắc đái thá

KeyboardInterrupt: ignored